In [2]:
#dict
data = open('words.txt', 'r')
words = []
for line in data:
    line = line.strip()
    if len(line) == 1:
        continue
    
    words.append(line)

print(len(words))

99118


In [9]:
from FST import FST as FST

palavras_de_teste = ['mon', 'thurs', 'tues', 'zon']

palavras_de_teste1 = ["A's", "AA's", "AB's", "ABM's", "AC's"]

palavras_de_teste2 = ['apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep']

palavras_de_teste3 = ['aab111','aab222','aab333','ab111', 'ab222', 'ab333']

fst = FST(words=palavras_de_teste)

In [10]:
fst.imprimir()
fst.imprimir_inverso()

Número de nós: 12


In [11]:
from graphviz import render

arquivo = 'fst.dot'
arquivo2 = 'fst_inverso.dot'
arquivo3 = 'trie.dot'

render('dot', 'png', arquivo)
render('dot', 'png', arquivo2)
# render('dot', 'png', arquivo3)

'fst_inverso.dot.png'

In [6]:
def get_matches(word):
    resp = fst.checar_combinacoes(word)
    #print("Total de palavras encontradas: " + str(len(resp)) + "\n")
    return resp

In [18]:
def get_matches_for(word):
    matches = []
    cont = 0
    for w in words:
        if w.startswith(word):
            matches.append(w)
            cont += 1
    #print("Total de palavras encontradas: " + str(len(matches)) + "\n")
    return matches

In [8]:
def get_matches_binary_search(word):
    num_words = len(words)
    matches = []
    cont = 0
    ini = 0
    fim = num_words - 1
    atual = 0
    
    while ini <= fim:
        atual = (ini + fim) // 2
        if (words[atual].startswith(word)):
            fim = -1
        elif (words[atual] > word):
            fim = atual - 1
        else:
            ini = atual + 1
    
    while words[atual-1].startswith(word):
        atual -= 1

    while words[atual].startswith(word) and atual < len(words)-1:
        matches.append(words[atual])
        cont += 1
        atual += 1

    if atual == len(words)-1:
        if words[atual].startswith(word):
            matches.append(words[atual])
            cont += 1

    #print("Total de palavras encontradas: " + str(len(matches)) + "\n")
    return matches

In [19]:
print(get_matches("Agnew"))

["Agnew's"]


In [20]:
print(get_matches_for("Agnew"))

['Agnew', "Agnew's"]


In [21]:
print(get_matches_binary_search("Agnew"))

['Agnew', "Agnew's"]


In [ ]:
prefixos = set()

for word in words:
    for i in range(1, len(word)):
        prefixo = word[:i]

        if prefixo not in prefixos:
            len1 = len(get_matches(prefixo))
            try:
                len2 = len(get_matches_binary_search(prefixo))
            except:
                print(prefixo, "erro")
            

            if len2 != len1:
                print(prefixo, len1, len2)

        prefixos.add(prefixo)



A 1322 1227
A' 2 1
AA 2 1
AA' 2 1
AB 4 2
AB' 2 1
ABM 2 1
ABM' 2 1
AC 4 2
AC' 2 1
ACT 2 1
ACTH 2 1
ACTH' 2 1
AI 4 2
AI' 2 1
AID 2 1
AIDS 2 1
AIDS' 2 1
AM 2 1
AM' 2 1
AO 3 2
AOL 3 2
AOL' 2 1
AS 4 2
ASC 2 1
ASCI 2 1
ASCII 2 1
ASCII' 2 1
ASL 2 1
ASL' 2 1
AT 4 2
ATM 2 1
ATM' 2 1
ATP 2 1
ATP' 2 1
AW 2 1
AWO 2 1
AWOL 2 1
AWOL' 2 1
AZ 4 2
AZ' 2 1
AZT 2 1
AZT' 2 1
Aa 5 4
Aal 3 2
Aali 3 2
Aaliy 3 2
Aaliya 3 2
Aaliyah 3 2
Aaliyah' 2 1
Ab 50 36
Abb 8 6
Abbo 3 2
Abbot 3 2
Abbott 3 2
Abbott' 2 1
Abby 3 2
Abby' 2 1
Abd 3 2
Abdu 3 2
Abdul 3 2
Abdul' 2 1
Abe 16 11
Abe' 2 1
Abel 7 5
Abel' 2 1
Abels 3 2
Abelso 3 2
Abelson 3 2
Abelson' 2 1
Aber 6 4
Aberd 3 2
Aberde 3 2
Aberdee 3 2
Aberdeen 3 2
Aberdeen' 2 1
Abern 3 2
Aberna 3 2
Abernat 3 2
Abernath 3 2
Abernathy 3 2
Abernathy' 2 1
Abi 5 4
Abid 3 2
Abidj 3 2
Abidja 3 2
Abidjan 3 2
Abidjan' 2 1
Abn 3 2
Abne 3 2
Abner 3 2
Abner' 2 1
Abr 7 5
Abra 7 5
Abrah 3 2
Abraha 3 2
Abraham 3 2
Abraham' 2 1
Abram 4 3
Abram' 2 1
Aby 6 4
Abys 6 4
Abyss 6 4
Abyssi 6 4
Abyss

In [ ]:
import tkinter as tk
import timeit

class AutocompleteText(tk.Text):
    def __init__(self, *args, **kwargs):
        self.callback = kwargs.pop("autocomplete", None)
        self.comparativo1 = kwargs.pop("comparativo1", None)
        super().__init__(*args, **kwargs)
        self.bind("<Any-KeyRelease>", self._autocomplete)
        self.bind("<Tab>", self._handle_tab)

    def _handle_tab(self, event):
        tag_ranges= self.tag_ranges("autocomplete")
        if tag_ranges:
            self.mark_set("insert", tag_ranges[1])
            self.tag_remove("sel", "1.0", "end")
            self.tag_remove("autocomplete", "1.0", "end")

            return "break"

    def _autocomplete(self, event):
        if event.char and self.callback:
            word = self.get("insert-1c wordstart", "insert-1c wordend")
            inicio = timeit.default_timer()
            matches = self.callback(word)
            fim = timeit.default_timer()

            inicio2 = timeit.default_timer()
            matches2 = self.comparativo1(word)
            fim2 = timeit.default_timer()

            text_tempo1["text"] = "Tempo de atualização I: " + str(round((fim - inicio)*1000,3)) + " ms"

            text_tempo2["text"] = "Tempo de atualização II: " + str(round((fim2 - inicio2)*1000,3)) + " ms"

            if matches:
                remainder = matches[0][len(word):]
                insert = self.index("insert")
                self.insert(insert, remainder, ("sel", "autocomplete"))
                self.mark_set("insert", insert)
            
            texto = "Outras Sugestões: \n"
            for match in matches[1:]:
                texto += match + "\n"
            texto_outras_opcoes["text"] = texto
            

root = tk.Tk()

# titulo
root.title("Autocomplete")

# tamanho da janela
root.geometry("300x650")

# Label
label = tk.Label(root, text="Digite uma palavra: ")
label.pack(padx=10, pady=10)

# texto
text = AutocompleteText(root, autocomplete=get_matches, comparativo1 = get_matches_binary_search)
text.pack(padx=10, pady=10)

# Outras opções
texto_outras_opcoes = tk.Label(root, text="Outras opções: ")
texto_outras_opcoes.pack(padx=10, pady=10)

# tempo de atualização
text_tempo1 = tk.Label(root, text="Tempo de atualização I: ")
text_tempo1.pack(padx=10, pady=5)

# tempo de atualização
text_tempo2 = tk.Label(root, text="Tempo de atualização II: ")
text_tempo2.pack(padx=10, pady=5)

root.mainloop()

Total de palavras encontradas: 4666

Total de palavras encontradas: 4666

Total de palavras encontradas: 6771

Total de palavras encontradas: 6771

Total de palavras encontradas: 1729

Total de palavras encontradas: 1729

Total de palavras encontradas: 199

Total de palavras encontradas: 199

Total de palavras encontradas: 34

Total de palavras encontradas: 34

Total de palavras encontradas: 14

Total de palavras encontradas: 13

Total de palavras encontradas: 4

Total de palavras encontradas: 4



In [ ]:
class Teste:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __eq__(self, other):
        return self.x == other.x

    def __hash__(self):
        return hash((self.x, self.y))

# Exemplo de uso
objeto1 = Teste(1, 2)
objeto2 = Teste(1, 2)
objeto3 = Teste(1, 4)

conjunto = set([objeto1, objeto2, objeto3])

print(len(conjunto))

print(objeto1 == objeto3)

2
True
